Fitting Multiple Change-point Models for Time Series Data
----
Fan Li (fl53)

### Background


Change-point models are proposed to detect possible heterogeneity in a stochatic process. Formally, suppose we have observed a set observations indexed by time $\{X\}_{t=1}^T$, where $\{X\}_{t=1}^{t_k}$ are identically distributed according to $\mathcal{F}_0$, and $\{X\}_{t=t_k+1}^T$ by $\mathcal{F}_1$. If $\mathcal{F}_0\neq \mathcal{F}_1$, we call the time point $t=t_k$ a change point, which is unknown and of interest in practice. In the above process, only a single change point exists. In other complex models, there could be more than one such change points underlying the heterogeneity. A major focus of change-point models is to determine the totality of such change-points, revealing the dynamics of an ongoing process. The application of change-point models encompass a broad spectrum, such as control theory, time series and reliability analysis.

Based on how the observations are collected, change-point models are classified as sequential and posteriori detection models. Sequential models updates the estimates along with the collection, while the posteriori models analyzed the data post to the collection. Based on model assumptions, we have parametric and non-parametric detection models. Further, models are classified into dicrete- and continuous-time (state) detection models, by the nature of the stochastic process. In this project, we'll mainly looks at the discrete-time discrete-state Markov process with a multiple change-point posetriori detection model. To fix ideas, we will only consider parametric models. The extensions to non-parametric models are natural but require more extensive computation. 

We state the model formally as the following. Suppose we observe a time series $Y_n=\{y_1,\ldots,y_n\}$ where the distribution of $y_t|Y_{t-1}$ depends on the unknown parameter $\xi_t$. Given a collection of unknown time of change-point $\Upsilon_m=\{\tau_1,\ldots,\tau_m\}$ and let $\tau_0=0$, $\tau_{m+1}=n$, we further assume the regime $\xi_t$ depends on the the interval determined by consectutive change-point time:

$$\xi_t=\begin{cases} \theta_1 &\mbox{if } \tau_0 < t \leq \tau_1, \\ 
\theta_2 &\mbox{if } \tau_1 < t \leq \tau_2,\\
\vdots &\vdots~~~~\vdots\\
\theta_m &\mbox{if } \tau_{m-1} < t \leq \tau_m,\\
\theta_{m+1} &\mbox{if } \tau_{m} < t \leq \tau_{m+1}.\end{cases}$$

Further let $Y_t=(y_1,\ldots,y_t)$ indicate the history up to time $t$ and $Y^{t,t'}=(y_t,\ldots,y_{t'})$ the history from $t$ to $t'$, then the joint likelihood of the time series is expressed as
$$\mathcal{L}(Y_n|\Theta,\Upsilon_m)=\prod_{k=1}^{m+1}f(Y^{\tau_{k-1}+1,\tau_k}|Y_{\tau_{k-1},\theta_k,\tau_k}).$$

Given the prior $\pi(\Theta,\Upsilon)$, the posterior inference can be carried out via Morkov Chain Monte Carlo (MCMC). However, the conventional method which samples $\tau_k$ one at a time is far from ideal due to non-existence of appropriate proposal densities. And the associated timing of computation is also an issue. The method proposed by Chib utilized a new parameterization that reproduces the exact same model with a set of augmented latent state variable (regime) $S_n=\{s_t\}_{t=1}^n\in\{1,\ldots,m+1\}^n$. Namely, the regime variable $s_t=k$ indicates that the generating distribution of $y_t$ is $f(y_t|Y_{t-1},\theta_k)$. Resembling the idea of a hidden Markov model (HMM), we model the state variables with a one-step ahead transition probability matrix 

$$P=\left(\begin{array}{cccccc}
p_{11} & p_{12} & 0 & \ldots & 0 \\
0 & p_{22} & p_{23} & \ldots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
\ldots & 0 & 0 & p_{mm} & p_{m,m+1} \\
0 & 0 & \ldots & 0 & 1 \end{array} \right)$$,

where $p_{ij}=\mathcal{P}(s_t=j|s_t=i)$ as in a conventional Markov process.

### Outline of the Pseudo-code

With data augmentation, the posterior inference becomes more tractable. We proceed with the following order:
- Update transition matrix $P|S_n$:

First note that there is only a single unknown parameter in each row of $P$, say the diagonal element $p_{ii}$. Suppose we assign conjugate Beta$(a,b)$ prior independently for $p_{ii}$, where $a,b$ are chosen according to prior belief about the mean duration of each regime. The full conditional is given by 

$$p_{ii}|\cdot \sim \text{Beta}(a+n_{ii},b+n_{i,i+1}),~~~~\forall i\in \{1,\ldots,m\},$$

where $n_{ij}$ is the number of one-step transitions from state $i$ to state $j$ in the current sequence $S_n$.

- Update the model parameters $\Theta|Y_n,S_n,P$:

Since we are dealing with parametric models, so $\Theta$ is model-specific. We will present more details on derivation of the full conditionals in subsequent sections for specific models.

- Update the augmented state variables $S_n$:

Let $S_t=(s_1\ldots,s_t)$ denote the state history up to time $t$ and $S^{t+1}=\{s_{t+1,\ldots,s_{n}}\}$ the history from time $t+1$ until the end, and similarly for $Y_t$ and $Y^{t+1}$. The only way shown (Chib 1993) to derive the sampler with closed-form expressions is to write the joint posterior full conditional in the reverse order as:

$$\prod_{t=1}^{n-1} p(s_t|Y_n,S^{t+1},\Theta,P).$$

Fixing $s_n=m+1$, the sampler generates $s_t$ from $p(s_t|Y_n,S^{t+1},\Theta,P)$ from $t=n-1$ until $t=1$. Chib showed that 

$$p(s_t|Y_n,S^{t+1},\Theta,P)\propto p(s_t|Y_t,\Theta,P)p(s_{t+1}|s_t,P),$$

where the first item is the univariate conditional posterior of the state variable at time $t$, and the last item is the transition probability directly available from the current update. Note that $p(s_t|Y_n,S^{t+1},\Theta,P)$ only have point masses on the consecutive pairs given the value of $s_{t+1}$, so the calculation of the normalization constant is straightforward. Further, given $p(s_{t-1}=l|Y_{t-1},\Theta,P)$,

$$p(s_t=k|Y_t,\Theta,P)=\frac{p(s_t=k|Y_{t-1},\Theta,P)f(y_t|Y_{t-1},\theta_k)}{\sum_{l=k-1}^k p(s_t=l|Y_{t-1},\Theta,P)f(y_t|Y_{t-1},\theta_l)},$$

where
\begin{equation}
p(s_t=l|Y_{t-1},\Theta,P)=\sum_{l=k-1}^k p_{lk}p(s_{t-1}=l|Y_{t-1},\Theta,P),
\end{equation}

for $k=1,\ldots,m$.

### Outline of the Unit Test (for a single update)

- Test for updating transition probability $p_{ii}$ - assess whether each update is a valid posterior probability mass between $(0,1)$;

- Test for updating $\theta$ - since the model parameter is model-specific, we use the binomial example for illustration. 

 - Make sure that the binomial probabilities are also within $(0,1)$.

- Test for updating the latent states $s_{t}$ for each $t$:

 - For the recursive calculation of quantities (like the HMM forward-backward steps), check whether the initiated lag-1 conditional mass satisfy
   
   $$\sum_{l=1}^{m+1} p(s_{t-1}=l|Y_{t-1},\Theta,P)=1~~\forall~t\in\{2,\ldots,n+1\},$$
   
 - Further check the lag-0 conditional mass
 
   $$\sum_{k=1}^{m+1} p(s_t=k|Y_t,\Theta,P)=1~~\forall~t\in\{1,\ldots,n\},$$
  
 - Lastly, check whether the updated sequence $\{s_t\}$ is properly ordered such that
   
   $$s_{t-1} \leq s_{t}~~\forall~t\in \{2,\ldots,n\}$$


----
This project is based on the paper "Estimation and comparison pf multiple change-point models" by Chib (1998). 